# 🚀 Fine-tune PhoBERT cho Travel Sentiment Analysis

**Thời gian chạy: ~5-10 phút với GPU T4**

## Hướng dẫn:
1. Vào **Runtime > Change runtime type > GPU (T4)**
2. Chạy từng cell từ trên xuống
3. Download model về máy ở cuối notebook

## 1️⃣ Cài đặt thư viện

In [ ]:
# Cài đặt phiên bản mới nhất của transformers
!pip install -q transformers>=4.30.0 datasets scikit-learn pandas accelerate

In [ ]:
import torch
import transformers
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Check if transformers version is compatible
from packaging import version
if version.parse(transformers.__version__) < version.parse("4.30.0"):
    print("\n⚠️  Transformers version is old. Upgrading...")
    !pip install -q --upgrade transformers
    print("✅ Please restart runtime: Runtime > Restart runtime")

## 2️⃣ Tạo Training Dataset

Dataset bao gồm:
- **35% Positive**: Câu khen rõ ràng
- **35% Negative**: Câu chê rõ ràng
- **30% Neutral/Mixed**: Câu trung lập, mixed sentiment (QUAN TRỌNG!)

In [ ]:
import random
import pandas as pd
from typing import List, Dict

# Word banks
WORD_BANKS = {
    'adj_pos': ['đẹp', 'tuyệt vời', 'xuất sắc', 'tốt', 'ổn', 'ok', 'xịn', 'đỉnh',
                'thích', 'hay', 'ấn tượng', 'mê', 'thơ mộng', 'lãng mạn'],
    'adj_neg': ['tệ', 'dở', 'kém', 'xấu', 'chán', 'thất vọng', 'tồi', 'bẩn'],
    'service_pos': ['nhân viên thân thiện', 'phục vụ tốt', 'hỗ trợ nhiệt tình',
                    'dịch vụ chuyên nghiệp', 'nhân viên vui vẻ'],
    'service_neg': ['nhân viên thái độ kém', 'phục vụ chậm', 'không nhiệt tình',
                    'dịch vụ tệ', 'nhân viên cộc lốc'],
    'facility_pos': ['phòng sạch sẽ', 'tiện nghi đầy đủ', 'wifi mạnh',
                     'phòng rộng rãi', 'decor đẹp'],
    'facility_neg': ['phòng bẩn', 'cơ sở vật chất cũ', 'wifi yếu',
                     'phòng nhỏ', 'xuống cấp'],
    'price_neg': ['đắt', 'mắc', 'cao', 'chát', 'không đáng tiền'],
    'hygiene_neg': ['bẩn quá', 'wc hôi', 'rác nhiều', 'mùi khó chịu'],
    'crowd_neg': ['đông quá', 'chen chúc', 'chờ lâu', 'ồn ào', 'xếp hàng dài'],
}

# Templates
POSITIVE_TEMPLATES = [
    "Địa điểm {adj_pos}, {service_pos}, rất recommend!",
    "View {adj_pos}, phong cảnh {adj_pos}, đáng đi lắm!",
    "{service_pos}, {facility_pos}, sẽ quay lại!",
    "Tuyệt vời! {adj_pos}, {adj_pos}, 10 điểm!",
    "Cảnh {adj_pos}, chụp hình {adj_pos}, thích lắm!",
    "Địa điểm {adj_pos}, {service_pos}.",
    "View {adj_pos}, đáng để ghé thăm.",
    "{facility_pos}, {adj_pos}.",
]

NEGATIVE_TEMPLATES = [
    "Dịch vụ {adj_neg}, {service_neg}, không bao giờ quay lại!",
    "Giá {price_neg}, {adj_neg}, thất vọng!",
    "{hygiene_neg}, {adj_neg}, không recommend!",
    "Quá {adj_neg}, {service_neg}, phí tiền!",
    "{crowd_neg}, {service_neg}, {adj_neg}!",
    "Hơi {adj_neg}, {service_neg}.",
    "{adj_neg}, {crowd_neg}.",
]

# NEUTRAL/MIXED - QUAN TRỌNG NHẤT
NEUTRAL_TEMPLATES = [
    "Cảnh {adj_pos} nhưng {adj_neg}.",
    "View {adj_pos}, nhưng {service_neg}.",
    "{facility_pos} nhưng {price_neg}.",
    "{adj_pos} nhưng {crowd_neg}.",
    "Tuy {adj_neg} nhưng {adj_pos}.",
    "Mặc dù {price_neg}, nhưng {adj_pos}.",
    "Cũng được, không có gì đặc biệt.",
    "Tạm ổn, bình thường.",
    "Khá ok, không quá {adj_pos}.",
    "Hơi {adj_neg}, còn lại ok.",
    "Không {adj_neg} lắm, cũng được.",
    "Giá hơi {price_neg}, còn lại ok.",
    "{adj_pos}, chỉ hơi {adj_neg} thôi.",
]

# Edge cases - nhân 20 lần để tăng weight
EDGE_CASES = [
    {"text": "Không tệ", "label": "NEU"},
    {"text": "Không dở lắm", "label": "NEU"},
    {"text": "Cũng không tệ", "label": "NEU"},
    {"text": "Không đẹp", "label": "NEG"},
    {"text": "Không tốt lắm", "label": "NEG"},
    {"text": "Đẹp nhưng đắt", "label": "NEU"},
    {"text": "Rẻ nhưng dở", "label": "NEU"},
    {"text": "View đẹp nhưng xa", "label": "NEU"},
    {"text": "Hơi đắt", "label": "NEU"},
    {"text": "Khá ổn", "label": "NEU"},
    {"text": "Tạm được", "label": "NEU"},
    {"text": "Ok thôi", "label": "NEU"},
    {"text": "Bình thường", "label": "NEU"},
    {"text": "Cũng được", "label": "NEU"},
    {"text": "Giá hơi cao", "label": "NEU"},
    {"text": "Hơi đông", "label": "NEU"},
    {"text": "Hơi đắt nhưng đáng tiền", "label": "POS"},
    {"text": "Giá đắt nhưng đáng tiền", "label": "POS"},
    {"text": "Dep lam, rat thich", "label": "POS"},
    {"text": "Te qua, ko bao gio quay lai", "label": "NEG"},
    {"text": "Cung dc, binh thuong", "label": "NEU"},
]

def fill_template(template: str) -> str:
    result = template
    for key, words in WORD_BANKS.items():
        placeholder = '{' + key + '}'
        while placeholder in result:
            result = result.replace(placeholder, random.choice(words), 1)
    return result

def generate_dataset(num_samples: int = 3000):
    data = []

    # Positive (35%)
    for _ in range(int(num_samples * 0.35)):
        text = fill_template(random.choice(POSITIVE_TEMPLATES))
        data.append({'text': text, 'label': 'POS'})

    # Negative (35%)
    for _ in range(int(num_samples * 0.35)):
        text = fill_template(random.choice(NEGATIVE_TEMPLATES))
        data.append({'text': text, 'label': 'NEG'})

    # Neutral/Mixed (30%)
    for _ in range(int(num_samples * 0.30)):
        text = fill_template(random.choice(NEUTRAL_TEMPLATES))
        data.append({'text': text, 'label': 'NEU'})

    # Add edge cases (x20)
    for _ in range(20):
        data.extend(EDGE_CASES.copy())

    random.shuffle(data)
    return data

# Generate dataset
print("Generating dataset...")
data = generate_dataset(3000)
df = pd.DataFrame(data)

# Label mapping
LABEL_MAP = {'NEG': 0, 'NEU': 1, 'POS': 2}
df['label_id'] = df['label'].map(LABEL_MAP)

# Split
train_df = df.sample(frac=0.8, random_state=42)
remaining = df.drop(train_df.index)
val_df = remaining.sample(frac=0.5, random_state=42)
test_df = remaining.drop(val_df.index)

print(f"\n✅ Dataset created!")
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
print(f"\nLabel distribution (Train):")
print(train_df['label'].value_counts())

## 3️⃣ Load PhoBERT Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "wonrax/phobert-base-vietnamese-sentiment"
ID2LABEL = {0: "NEG", 1: "NEU", 2: "POS"}
LABEL2ID = {"NEG": 0, "NEU": 1, "POS": 2}

print("Loading PhoBERT model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=ID2LABEL,
    label2id=LABEL2ID,
    ignore_mismatched_sizes=True,
)

print(f"✅ Model loaded! Parameters: {model.num_parameters():,}")

## 4️⃣ Prepare Data cho Training

In [ ]:
from datasets import Dataset

MAX_LENGTH = 128

def create_dataset(df):
    dataset = Dataset.from_pandas(df[['text', 'label_id']])
    dataset = dataset.rename_column("label_id", "labels")

    def tokenize(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True,
            max_length=MAX_LENGTH,
        )

    return dataset.map(tokenize, batched=True, remove_columns=["text"])

train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)
test_dataset = create_dataset(test_df)

print(f"✅ Datasets prepared!")
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

## 5️⃣ Fine-tune Model 🚀

**Thời gian: ~5-10 phút với GPU T4**

In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro', zero_division=0
    )
    _, _, f1_per_class, _ = precision_recall_fscore_support(
        labels, predictions, average=None, zero_division=0
    )

    return {
        "accuracy": accuracy,
        "f1_macro": f1,
        "f1_neg": f1_per_class[0],
        "f1_neu": f1_per_class[1],
        "f1_pos": f1_per_class[2],
    }

# Training arguments - Compatible với cả phiên bản cũ và mới
training_args = TrainingArguments(
    output_dir="./phobert-travel-sentiment",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    eval_strategy="epoch",  # Thay đổi từ evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,
    logging_steps=50,
    report_to="none",
    fp16=True,  # Use FP16 for faster training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

print("🚀 Starting training...")
print(f"   Epochs: 3")
print(f"   Batch size: 16")
print(f"   Learning rate: 2e-5")
print()

trainer.train()

## 6️⃣ Evaluate on Test Set

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("📊 Evaluating on test set...")
results = trainer.evaluate(test_dataset)

print("\n" + "="*50)
print("TEST RESULTS")
print("="*50)
print(f"Accuracy:  {results['eval_accuracy']:.4f}")
print(f"F1 Macro:  {results['eval_f1_macro']:.4f}")
print(f"F1 NEG:    {results['eval_f1_neg']:.4f}")
print(f"F1 NEU:    {results['eval_f1_neu']:.4f}")
print(f"F1 POS:    {results['eval_f1_pos']:.4f}")

# Confusion matrix
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

print("\nConfusion Matrix:")
print(confusion_matrix(labels, preds))
print("\nClassification Report:")
print(classification_report(labels, preds, target_names=['NEG', 'NEU', 'POS']))

## 7️⃣ Test với các câu mẫu

In [ ]:
# Test samples
test_texts = [
    "Địa điểm rất đẹp, view tuyệt vời!",
    "Dịch vụ tệ, không bao giờ quay lại!",
    "Cũng được, không có gì đặc biệt.",
    "Cảnh đẹp nhưng đông quá.",
    "Không tệ lắm, tạm ổn.",
    "Hơi đắt nhưng đáng tiền.",
    "Ok thôi, bình thường.",
    "Tạm ổn, giá hơi cao.",
    "View đẹp, nhưng xa trung tâm.",
    "Phòng sạch, nhưng wifi yếu.",
]

model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("\n" + "="*60)
print("SAMPLE PREDICTIONS")
print("="*60)

for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LENGTH)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]

    pred_label = ID2LABEL[np.argmax(probs)]
    print(f"\n'{text}'")
    print(f"  → {pred_label} (NEG:{probs[0]:.2f}, NEU:{probs[1]:.2f}, POS:{probs[2]:.2f})")

## 8️⃣ Save & Download Model

In [ ]:
# Save model
OUTPUT_DIR = "./phobert-travel-sentiment-final"
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Model saved to {OUTPUT_DIR}")

In [ ]:
# Zip model để download
!zip -r phobert-travel-sentiment.zip ./phobert-travel-sentiment-final

print("\n✅ Model zipped! Click link below to download:")

In [ ]:
# Download link
from google.colab import files
files.download('phobert-travel-sentiment.zip')

## 9️⃣ Hướng dẫn tích hợp vào Project

1. **Download** file `phobert-travel-sentiment.zip`

2. **Giải nén** vào thư mục project:
```
WebDuLich-fix-conflic/
└── travel/
    └── models/
        └── phobert-travel-sentiment/
            ├── config.json
            ├── model.safetensors
            ├── tokenizer.json
            └── ...
```

3. **Update `ai_engine.py`**:
```python
# Trong hàm load_model():
finetuned_path = os.path.join(settings.BASE_DIR, 'travel', 'models', 'phobert-travel-sentiment')
if os.path.exists(finetuned_path):
    model_name = finetuned_path
else:
    model_name = "wonrax/phobert-base-vietnamese-sentiment"
```

4. **Test**:
```bash
python test_comprehensive.py
```